In [ ]:
# Step 1: Data Preprocessing, Cleaning, and Merging of Stock Returns and Sector Data

import pandas as pd
import numpy as np
import os

# File Locations
#Input file paths
SECTOR_INFO_FILE_PATH = 'Sector Information.xlsm'  # Original sector info
STOCK_RETURN_FILE_PATH = 'Stock_Return_Data2023Fwd.xlsx'
STOCK_VARREF_FILE_PATH = 'Financial Ratios_Variable_Reference.xlsx'
SIC_CODE_FILE_PATH = 'sic_codes_wikipedia.xlsx'

#Output file paths
SECTOR_INFO_wrds_FILE_PATH = 'sector_information_wrds.xlsx'  # WRDS sector info
SECTOR_INFO_UPDATED_FILE_PATH = 'sector_information_updated_with_wrds.xlsx'  # Output file
STOCK_DENG_OUT_FILE_PATH = 'Sector_Info_Return_Ratio_Merged.xlsx' # Final Output file


# Source Sector Information from Wharton Research Data Service (WRDS)

# Flag - Enable Sector Name retrieval from WRDS. Valid credentials required. Best Source for Sector Info
EXTRACT_wrds_SECTOR_INFO_FLAG = False

if EXTRACT_wrds_SECTOR_INFO_FLAG:
    SIC_CODE_FILE_PATH = 'sic_codes_wikipedia.xlsx'
    df_sic = pd.read_excel(SIC_CODE_FILE_PATH)
    # Connect to WRDS
    db = wrds.Connection()
    
    
    # SQL Query to get GVKEY, PERMNO, TICKER, and Sector Name
    sql_code="""
    SELECT DISTINCT
        CAST(c.gvkey AS INTEGER) AS "gvkey_wrds",
        CAST(l.lpermno AS INTEGER) AS "permno_wrds",
        c.conm AS "comnam_wrds",
        CASE 
            WHEN s.ticker = 'N/A' OR s.ticker IS NULL THEN ''  -- Set to empty string if 'N/A' or NULL
            ELSE s.ticker 
        END AS ticker_wrds,
        CAST(c.sic AS INTEGER) AS "sic_code_wrds",
        CASE 
            WHEN CAST(c.sic AS INTEGER) BETWEEN 100 AND 999 THEN 'Agriculture, Forestry, and Fishing'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 1000 AND 1499 THEN 'Mining'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 1500 AND 1799 THEN 'Construction'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 2000 AND 3999 THEN 'Manufacturing'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 4000 AND 4999 THEN 'Transportation, Communications, Utilities'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 5000 AND 5199 THEN 'Wholesale Trade'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 5200 AND 5999 THEN 'Retail Trade'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 6000 AND 6799 THEN 'Finance, Insurance, and Real Estate'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 7000 AND 8999 THEN 'Services'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 9100 AND 9729 THEN 'Public Administration'
            WHEN CAST(c.sic AS INTEGER) BETWEEN 9900 AND 9999 THEN 'Nonclassifiable'
            ELSE ''
        END AS "sic_division_wrds",
        CAST(c.gsector AS INTEGER) AS "sector_code_wrds",
        CASE 
            WHEN CAST(c.gsector AS INTEGER) = 10 THEN 'Energy'
            WHEN CAST(c.gsector AS INTEGER) = 15 THEN 'Materials'
            WHEN CAST(c.gsector AS INTEGER) = 20 THEN 'Industrials'
            WHEN CAST(c.gsector AS INTEGER) = 25 THEN 'Consumer Discretionary'
            WHEN CAST(c.gsector AS INTEGER) = 30 THEN 'Consumer Staples'
            WHEN CAST(c.gsector AS INTEGER) = 35 THEN 'Health Care'
            WHEN CAST(c.gsector AS INTEGER) = 40 THEN 'Financials'
            WHEN CAST(c.gsector AS INTEGER) = 45 THEN 'Information Technology'
            WHEN CAST(c.gsector AS INTEGER) = 50 THEN 'Communication Services'
            WHEN CAST(c.gsector AS INTEGER) = 55 THEN 'Utilities'
            WHEN CAST(c.gsector AS INTEGER) = 60 THEN 'Real Estate'
            ELSE ''
        END AS "sector_name_wrds",  
        CASE 
            WHEN CAST(c.gsector AS INTEGER) = 10 THEN 'Energy: Oil, gas, coal, and fuels'
            WHEN CAST(c.gsector AS INTEGER) = 15 THEN 'Materials: Chemicals, metals, mining, forestry'
            WHEN CAST(c.gsector AS INTEGER) = 20 THEN 'Industrials: Manufacturing, machinery, transportation'
            WHEN CAST(c.gsector AS INTEGER) = 25 THEN 'Consumer Discretionary: Automobiles, hotels, media'
            WHEN CAST(c.gsector AS INTEGER) = 30 THEN 'Consumer Staples: Food, beverages, household goods'
            WHEN CAST(c.gsector AS INTEGER) = 35 THEN 'Health Care: Pharmaceuticals, biotech, hospitals'
            WHEN CAST(c.gsector AS INTEGER) = 40 THEN 'Financials: Banks, insurance, investment services'
            WHEN CAST(c.gsector AS INTEGER) = 45 THEN 'Information Technology: Software, IT services, semiconductors'
            WHEN CAST(c.gsector AS INTEGER) = 50 THEN 'Communication Services: Media, telecom, entertainment'
            WHEN CAST(c.gsector AS INTEGER) = 55 THEN 'Utilities: Electric, gas, water utilities'
            WHEN CAST(c.gsector AS INTEGER) = 60 THEN 'Real Estate: REITs, property development'
            ELSE ''
        END AS "sector_description_wrds"  
    FROM comp.company AS c
    JOIN crsp.ccmxpf_lnkhist AS l
        ON c.gvkey = l.gvkey
        AND l.linktype IN ('LU', 'LC')
        AND l.linkprim IN ('P', 'C')
    JOIN crsp.msenames AS s
        ON l.lpermno = s.permno
    ORDER BY gvkey_wrds;
    """
    
    # Fetch data
    df_wrds = db.raw_sql(sql_code)
    df_wrds = df_wrds.merge(df_sic, left_on=['sic_code_wrds'], right_on=['sic_code'], how="left")
    df_wrds.rename(columns={'sic_industry': 'sic_industry_wrds'}, inplace=True)
    df_wrds.drop(columns=['sic_code'], inplace=True)

    df_wrds['ticker_wrds'] = df_wrds['ticker_wrds'].str.strip()
    df_wrds['ticker_wrds']=df_wrds['ticker_wrds'].replace('#N/A', '').replace('N/A','')
    df_wrds['ticker_wrds']=df_wrds['ticker_wrds'].replace('', np.nan)
     
    df_wrds = df_wrds.dropna(subset=['ticker_wrds'])
    
    # Save as Excel
    df_wrds.to_excel(SECTOR_INFO_wrds_FILE_PATH, index=False, sheet_name='sector data')
    
    print(f"Data saved as {SECTOR_INFO_wrds_FILE_PATH}")



# Read stock information (renamed to sector_info_orig_df)
sector_info_orig_df = pd.read_excel(SECTOR_INFO_FILE_PATH, sheet_name='sector data')

# Convert TICKER columns to uppercase and remove spaces to avoid case mismatch
sector_info_orig_df['TICKER'] = sector_info_orig_df['TICKER'].str.upper().str.replace(' ', '')

# Read sector information from WRDS
sector_info_wrds_df = pd.read_excel(SECTOR_INFO_wrds_FILE_PATH)

# Drop rows with missing values in 'permno' or 'ticker'
sector_info_wrds_df.dropna(subset=['permno_wrds', 'ticker_wrds', 'sector_name_wrds'], inplace=True)

# Drop duplicate rows
sector_info_wrds_df.drop_duplicates(inplace=True)

# Merge based on 'permno' to retrieve matching sector name and ticker
merged_df = pd.merge(
    sector_info_orig_df, 
    sector_info_wrds_df, 
    left_on=['permno', 'TICKER'], 
    right_on=['permno_wrds', 'ticker_wrds'], 
    how='left', 
    #suffixes=('', '_wrds')
)

# Fill missing sector names in original data with sector_name from new_entries
merged_df['Sector Name'] = merged_df.apply(
    lambda row: row['sector_name_wrds'] if pd.isna(row['Sector Name']) and pd.notna(row['sector_name_wrds']) else row['Sector Name'], axis=1
 )

# Fill missing sector names in original data with sector_name from new_entries
merged_df['TICKER'] = merged_df.apply(
    lambda row: row['ticker_wrds'] if pd.isna(row['TICKER']) and pd.notna(row['ticker_wrds']) else row['TICKER'], axis=1
)
merged_df['New'] = 'N' 

# Identify unique records in WRDS data that are missing in the original data
new_entries = sector_info_wrds_df[
    ~sector_info_wrds_df['ticker_wrds'].isin(merged_df['TICKER']) |
    ~sector_info_wrds_df['permno_wrds'].isin(merged_df['permno'])
]

new_entries = new_entries.dropna(subset=['permno_wrds', 'ticker_wrds', 'sector_name_wrds'])  # Ensure permno, ticker, and sector_name are non-null
new_entries['gvkey'] = new_entries['gvkey_wrds']
new_entries['permno'] = new_entries['permno_wrds']
new_entries['TICKER'] = new_entries['ticker_wrds']
new_entries['Sector Name'] = new_entries['sector_name_wrds']
new_entries['New'] = 'Y'

# Append the unique new entries to the merged dataframe
#new_entries.rename(columns={'ticker': 'ticker_wrd', 'sector_name': 'sector_name_wrd'}, inplace=True)
merged_df = pd.concat([merged_df, new_entries], ignore_index=True)

merged_df['TICKER']=merged_df['TICKER'].replace('#N/A', '').replace('N/A','')
merged_df['TICKER']=merged_df['TICKER'].replace('', np.nan)

merged_df['Sector Name']=merged_df['Sector Name'].replace('#N/A', '').replace('N/A','')
merged_df['Sector Name']=merged_df['Sector Name'].replace('', np.nan)

merged_df['ticker_wrds']=merged_df['ticker_wrds'].replace('#N/A', '').replace('N/A','')
merged_df['ticker_wrds']=merged_df['ticker_wrds'].replace('', np.nan)

merged_df['sector_name_wrds']=merged_df['sector_name_wrds'].replace('#N/A', '').replace('N/A','')
merged_df['sector_name_wrds']=merged_df['sector_name_wrds'].replace('', np.nan)

# Drop rows with missing values in 'permno' or 'ticker', 'sector_name_wrds'
merged_df.dropna(subset=['permno_wrds', 'ticker_wrds', 'sector_name_wrds'], inplace=True)

# Drop duplicate records
merged_df.drop_duplicates(subset=['permno_wrds','ticker_wrds'], inplace=True)

merged_df = merged_df.astype({col: 'Int64' for col in merged_df.select_dtypes('float').columns})

merged_df['PERMNO_TICKER_LABEL'] = merged_df['permno_wrds'].astype(str) + '-' + merged_df['ticker_wrds'].astype(str)

print('Ticker symbols cleaned')


# Save the updated dataframe to a new Excel file
merged_df.to_excel(SECTOR_INFO_UPDATED_FILE_PATH, index=False)


print(f"Updated stock information saved to '{SECTOR_INFO_UPDATED_FILE_PATH}'")

df_cleaned = merged_df


# Create Sector Info Lookup Table to Merge with Stock Revenue
df_cleaned_lkup = df_cleaned[['PERMNO_TICKER_LABEL', 'ticker_wrds', 'sector_name_wrds']].drop_duplicates().copy()
df_cleaned_lkup.rename(columns={'sector_name_wrds': 'Sector_Name'}, inplace=True)

# Move 'permno_ticker_lkup' to the first column


# Read Stock Return Excel file
# filename: Stock_Return_Data2023Fwd.xlsx
# Worsksheet: "data"
# columns: (PERMNO, date, YEAR, MONTH, SIC, Industry, Code, TICKER, COMNAM, 
#            Price, Index, Past, 1-Month, Ret, Past, 3-Month, Ret, Past, 
#            6-Month, Ret, Past, 12-Month, Ret, Forward, 1-Month, Ret,
#            Forward, 3-Month, Ret, Forward, 6-Month, Ret, PERMNO_DATE_LABEL)

STOCK_RETURN_FILE_PATH = 'Stock_Return_Data2023Fwd.xlsx'
df_re = pd.read_excel(STOCK_RETURN_FILE_PATH)

# Add PERMNO_TICKER_LABEL (PERMNO-TECKER) to Stock Return for merging with Stock Information
df_re['TICKER'] = df_re['TICKER'].str.strip()
if 'PERMNO_TICKER_LABEL' not in df_re.columns:
    df_re.loc[df_re[['PERMNO', 'TICKER']].notna().all(axis=1), 'PERMNO_TICKER_LABEL'] = (
        df_re['PERMNO'].astype(str) + '-' + df_re['TICKER'].astype(str)
    )


# drop unnamed column from stock returns dataset
if 'Unnamed: 8' in df_re.columns:
    df_re.drop(columns=['Unnamed: 8'], inplace=True)
    print(f"dropped blank unnamed column 'Unnamed: 8'")
else:
    print(f"unnamed column 'Unnamed: 8' already removed")

# Merge df_re with df_cleaned_lkup to add Sector Name only once based on PERMNO_TICKER_LABEL
if 'Sector_Name' not in df_re.columns:
    df_re = df_re.merge(
    df_cleaned_lkup[['PERMNO_TICKER_LABEL', 'Sector_Name']], 
    on='PERMNO_TICKER_LABEL', 
    how='left'
)
print(f'Sector name from [sector_information_updated] added to Stock Returns')

columns_to_fill_with_zeros = [
    'Past 1-Month Ret', 'Past 3-Month Ret', 'Past 6-Month Ret', 'Past 12-Month Ret',
    'Forward 1-Month Ret', 'Forward 3-Month Ret', 'Forward 6-Month Ret'
]

df_re[columns_to_fill_with_zeros] = (df_re[columns_to_fill_with_zeros]
    .apply(pd.to_numeric, errors='coerce')
    .astype(float)
    .fillna(0.000)
)

# Read Financial Ratios Data Excel file
# filename: Financial Ratios Data 2003_2023x.xlsm
# Worsksheet: "financial_ratios"
# columns: 
# 'gvkey', 'permno', 'adate', 'qdate', 'public_date', 'TICKER', 'year', 'month', 'PERMNO-YearMonth', 'CAPEI', 
# 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 
# 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity', 'pretret_noa', 
# 'pretret_earnat', 'GProf', 'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio', 
# 'int_debt', 'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at', 'debt_ebitda', 'short_debt', 
# 'curr_debt', 'lt_debt', 'profit_lct', 'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 
# 'debt_assets', 'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio', 'quick_ratio', 
# 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn', 'rect_turn', 'pay_turn', 'sale_invcap', 
# 'sale_equity', 'sale_nwc', 'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb'

STOCK_FINANCIAL_RATIOS_FILE_PATH = 'Financial Ratios Data 2003_2023x.xlsm'
df_fr = pd.read_excel(STOCK_FINANCIAL_RATIOS_FILE_PATH, skiprows=1)

# merge stock returns with financial ratios
df_fr['TICKER'] = df_fr['TICKER'].str.strip()
# Drop unwanted columns from df_fr

for col in ['gvkey', 'permno']:
    if col in df_fr.columns:
        df_fr.drop(columns=[col], inplace=True)

df_re_fr = df_re.merge(df_fr, left_on=['PERMNO_DATE_LABEL', 'TICKER'], right_on=['PERMNO-YearMonth', 'TICKER'], how='left')

# Exclude Records with Sector Name in Financials, 'Real Estate', 'Utilities' Per project Instruction
excluded_rec_count = df_re['Sector_Name'].isin(['Financials', 'Real Estate', 'Utilities']).sum()
print(f"\nDropped records in ['Financials', 'Real Estate', 'Utilities']: {excluded_rec_count}\n")

df_re_fr=df_re_fr[
    (~df_re_fr['Sector_Name'].isin(['Financials', 'Real Estate', 'Utilities'])) &  # Exclude sectors
    (df_re_fr['TICKER'].notna()) &  # Remove NaN or empty TICKER
    (df_re_fr['Sector_Name'].notna())]

# Identify TICKER groups where all values in a column are NaN

columns_to_interpolate = [
    'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf', 'npm',
    'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
    'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
    'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt', 'int_totdebt', 'cash_lt',
    'invt_act', 'rect_act', 'debt_at', 'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt',
    'profit_lct', 'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
    'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio', 'quick_ratio',
    'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn', 'rect_turn', 'pay_turn',
    'sale_invcap', 'sale_equity', 'sale_nwc', 'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb'
]
problematic_tickers = {}

for col in columns_to_interpolate:
    empty_groups = df_re_fr.groupby('TICKER')[col].apply(lambda x: x.isna().all())
    tickers_with_all_nan = empty_groups[empty_groups].index.tolist()
    
    if tickers_with_all_nan:
        problematic_tickers[col] = tickers_with_all_nan

print("Columns with all-NaN TICKER groups:")
for col, tickers in problematic_tickers.items():
    print(f"{col}: {tickers}")


# ### drop tickers with all NA financial ratios


# Define the list of tickers to drop
tickers_to_drop = {'AIZ', 'AQU', 'BF', 'CCL', 'DALN', 'DE', 'DHI', 'DISCA', 'DPZ', 'ESV',
                   'ET', 'ETE', 'GOOGL', 'HCP', 'IDNX', 'JXJT', 'KBH', 'KBSF', 'KMRT', 'LEN',
                   'MDR', 'MXB', 'NOW', 'NVR', 'ODETA', 'ODETB', 'PARA', 'PARAA', 'PCAR',
                   'PHM', 'PX', 'SLB', 'TNL', 'TUGC', 'TXT', 'UAA', 'UN', 'V', 'VIAC',
                   'VIACA', 'VTR', 'WBR'}

# Drop records with tickers in the list
df_re_fr = df_re_fr[~df_re_fr['TICKER'].isin(tickers_to_drop)]

# Reset index after filtering
df_re_fr = df_re_fr.reset_index(drop=True)


# Reset index to avoid multi-index issues
df_re_fr.reset_index(inplace=True)

# Convert public_date to datetime and create YEAR column
df_re_fr['public_date'] = pd.to_datetime(df_re_fr['public_date'])

# Set index properly for sorting
df_re_fr = df_re_fr.set_index(['PERMNO_TICKER_LABEL', 'public_date']).sort_index(level=['PERMNO_TICKER_LABEL', 'public_date'])

# List of financial ratios to interpolate
columns_to_interpolate = [
    'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf', 'npm',
    'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
    'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
    'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt', 'int_totdebt', 'cash_lt',
    'invt_act', 'rect_act', 'debt_at', 'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt',
    'profit_lct', 'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
    'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio', 'quick_ratio',
    'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn', 'rect_turn', 'pay_turn',
    'sale_invcap', 'sale_equity', 'sale_nwc', 'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb'
]

# Function to apply hybrid imputation within each TICKER group
def impute_financial_ratios(group):
    """Hybrid Imputation for Financial Metrics"""
    group[columns_to_interpolate] = group[columns_to_interpolate].interpolate(method='linear', limit_direction='both')

    # Only apply rolling median where at least one non-null value exists
    for col in columns_to_interpolate:
        if not group[col].isnull().all():  # Check if the column has any non-null values
            group[col] = (
                group[col]
                .fillna(group[col].rolling(3, min_periods=1).median())
                .fillna(group[col].rolling(6, min_periods=1).median())
                .fillna(group[col].rolling(9, min_periods=1).median())
                .fillna(group[col].rolling(12, min_periods=1).median())
            )

    # Final backfill & forward fill
    group[columns_to_interpolate] = group[columns_to_interpolate].bfill().ffill()

    return group

# Apply per TICKER
df_re_fr = df_re_fr.groupby(level='PERMNO_TICKER_LABEL', group_keys=False).apply(impute_financial_ratios)

# Final step: Fill remaining missing values using YEAR and TICKER median
df_re_fr = df_re_fr.reset_index()
df_re_fr[columns_to_interpolate] = df_re_fr.groupby(['PERMNO_TICKER_LABEL'])[columns_to_interpolate].transform(lambda x: x.fillna(x.median(skipna=True)))

# Reset index
df_re_fr.reset_index(inplace=True)


# Save merged datset to excel file. Can continue analysis with df_re

# Export file for merged Stock Returns and Sector Name dataset

# Check if the file exists
if os.path.isfile(STOCK_DENG_OUT_FILE_PATH):
    # The file exists
    print(f'Deleting File {STOCK_DENG_OUT_FILE_PATH} and resaving')
    os.remove(STOCK_DENG_OUT_FILE_PATH) 
    
else:
# The file does not exist
    print(f'Saving New File to {STOCK_DENG_OUT_FILE_PATH}')

df_re_fr.to_excel(STOCK_DENG_OUT_FILE_PATH, index=False)
print(f'File Saved')
